In [1]:
import pandas as pd

df = pd.read_csv('customer_data.csv')

df.head()


,id,age,gender,income,education,region,loyalty_status,purchase_frequency,purchase_amount,product_category,promotion_usage,satisfaction_score
0,1,27,Male,40682,Bachelor,East,Gold,frequent,18249,Books,0,6
1,2,29,Male,15317,Masters,West,Regular,rare,4557,Clothing,1,6
2,3,37,Male,38849,Bachelor,West,Silver,rare,11822,Clothing,0,6
3,4,30,Male,11568,HighSchool,South,Regular,frequent,4098,Food,0,7
4,5,31,Female,46952,College,North,Regular,occasional,19685,Clothing,1,5


In [2]:
# import pandas as pd
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score



# # Encoding categorical variables
# label_encoders = {}
# categorical_columns = ['gender', 'education', 'region', 'loyalty_status', 'purchase_frequency', 'product_category']

# for column in categorical_columns:
#     le = LabelEncoder()
#     df[column] = le.fit_transform(df[column])
#     label_encoders[column] = le

# # Selecting features and target variable
# X = df.drop(columns=['id', 'purchase_amount'])
# y = df['purchase_amount']

# # Splitting the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Initializing and training the model
# model = RandomForestRegressor(random_state=42)
# model.fit(X_train, y_train)

# # Making predictions
# y_pred = model.predict(X_test)

# # Evaluating the model
# mae = mean_absolute_error(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f"Mean Absolute Error (MAE): {mae}")
# print(f"Mean Squared Error (MSE): {mse}")
# print(f"R-squared (R2): {r2}")


In [3]:
df['purchase_amount'].max() - df['purchase_amount'].min()

25086

# Grid Search to Determine Best Hyperparameters

In [4]:
# from sklearn.model_selection import GridSearchCV

# # Define the parameter grid
# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [None, 10, 20, 30],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'bootstrap': [True, False]
# }

# # Initialize the Random Forest model
# rf = RandomForestRegressor(random_state=42)

# # Initialize GridSearchCV
# grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
#                            cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')

# # Fit GridSearchCV
# grid_search.fit(X_train, y_train)

# # Get the best parameters and best estimator
# best_params = grid_search.best_params_
# best_estimator = grid_search.best_estimator_

# # Make predictions with the best estimator
# y_pred_best = best_estimator.predict(X_test)

# # Evaluate the model
# mae_best = mean_absolute_error(y_test, y_pred_best)
# mse_best = mean_squared_error(y_test, y_pred_best)
# r2_best = r2_score(y_test, y_pred_best)

# print(f"Best Parameters: {best_params}")
# print(f"Mean Absolute Error (MAE) after tuning: {mae_best}")
# print(f"Mean Squared Error (MSE) after tuning: {mse_best}")
# print(f"R-squared (R2) after tuning: {r2_best}")


# Re-run Model with Best Hyperparameters

In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Assuming df is your DataFrame
# Encoding categorical variables
label_encoders = {}
categorical_columns = ['gender', 'education', 'region', 'loyalty_status', 'purchase_frequency', 'product_category']

for column in categorical_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Selecting features and target variable
X = df.drop(columns=['id', 'purchase_amount'])
y = df['purchase_amount']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initializing and training the RandomForestRegressor with the best parameters
best_params = {'bootstrap': True, 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
model = RandomForestRegressor(**best_params, random_state=42)
model.fit(X_train, y_train)

# Making predictions
y_pred = model.predict(X_test)

# Evaluating the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")


Mean Absolute Error (MAE): 1107.532717423373
Mean Squared Error (MSE): 2351542.961364477
R-squared (R2): 0.8974216514952844
